In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Kaitangata,NZ,2022-04-04 01:30:05,-46.2817,169.8464,63.57,56,3,4.83
1,1,Arraial Do Cabo,BR,2022-04-04 01:28:15,-22.9661,-42.0278,76.57,87,3,4.38
2,2,Airai,TL,2022-04-04 01:30:06,-8.9266,125.4092,73.18,51,9,2.48
3,3,Villa Bruzual,VE,2022-04-04 01:30:07,9.3328,-69.1208,82.09,54,51,2.59
4,4,Port Alfred,ZA,2022-04-04 01:30:07,-33.5906,26.8910,66.18,89,50,7.96


In [3]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [5]:
!jupyter nbextension enable --py --sys-prefix gmaps
!jupyter nbextension enable --py --sys-prefix widgetsnbextension

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [6]:
# Get the maximum temperature.
#max_temp = city_data_df["Max Temp"]
#temps = []
#for temp in max_temp:
    #temps.append(max(temp, 0))

In [8]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp])
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [92]:
#Adjust Heatmap Zoom, Intensity, and Point Radius
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

In [9]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [13]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)


,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
1,1,Arraial Do Cabo,BR,2022-04-04 01:28:15,-22.9661,-42.0278,76.57,87,3,4.38
3,3,Villa Bruzual,VE,2022-04-04 01:30:07,9.3328,-69.1208,82.09,54,51,2.59
6,6,Rikitea,PF,2022-04-04 01:28:57,-23.1203,-134.9692,77.43,65,100,9.60
10,10,Butaritari,KI,2022-04-04 01:29:06,3.0707,172.7902,81.39,72,97,11.92
12,12,Kavieng,PG,2022-04-04 01:28:46,-2.5744,150.7967,84.07,68,100,5.21
15,15,Hilo,US,2022-04-04 01:30:12,19.7297,-155.0900,78.53,80,100,12.66
19,19,Fomboni,KM,2022-04-04 01:30:15,-12.2800,43.7425,82.40,74,16,7.78
33,33,Alofi,NU,2022-04-04 01:29:06,-19.0595,-169.9187,80.49,74,85,5.75
38,38,Georgetown,MY,2022-04-04 01:29:39,5.4112,100.3354,82.33,91,20,4.61
44,44,Souillac,MU,2022-04-04 01:31:31,-20.5167,57.5167,79.18,83,40,11.50


In [14]:
preferred_cities_df.count()

City_ID       165
City          165
Country       165
Date          165
Lat           165
Lng           165
Max Temp      165
Humidity      165
Cloudiness    165
Wind Speed    165
dtype: int64

In [15]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Arraial Do Cabo,BR,76.57,-22.9661,-42.0278,
3,Villa Bruzual,VE,82.09,9.3328,-69.1208,
6,Rikitea,PF,77.43,-23.1203,-134.9692,
10,Butaritari,KI,81.39,3.0707,172.7902,
12,Kavieng,PG,84.07,-2.5744,150.7967,
15,Hilo,US,78.53,19.7297,-155.0900,
19,Fomboni,KM,82.40,-12.2800,43.7425,
33,Alofi,NU,80.49,-19.0595,-169.9187,
38,Georgetown,MY,82.33,5.4112,100.3354,
44,Souillac,MU,79.18,-20.5167,57.5167,


In [23]:
#Retrieve Hotels from a Nearby Search
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [24]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [28]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

Figure(layout=FigureLayout(height='420px'))